# kaggle Data Sets

Combining public country statistics from th US Census Bureau with the Human Freedom Index

### United States International Census (Google BigQuery)

* kaggle https://www.kaggle.com/census/census-bureau-international/metadata
* BigQuery https://console.cloud.google.com/bigquery?filter=solution-type:dataset&q=census&id=5d64defd-b1cf-40e6-ae6b-fd3558e15d00&subtask=details&subtaskValue=united-states-census-bureau%2Finternational-census-data&project=bootcamp-1569375678077&folder&organizationId&subtaskIndex=3

### Human Freedom Index (csv)
* kaggle https://www.kaggle.com/gsutters/the-human-freedom-index

ETL Repository https://github.com/Jmaddalena42/ETL_Project

## Obtain Census Dataset

BigQuery Public Datasets https://cloud.google.com/bigquery/public-data/

### Enable Google Cloud services

Google Cloud Python Service Quick Start https://cloud.google.com/bigquery/docs/quickstarts/quickstart-client-libraries#client-libraries-install-python

### Install Dependencies

$ pip install google.cloud

$ pip install --upgrade google-cloud-bigquery

$ pip install --upgrade google-cloud-storage

### Get Service Account API Credentials

Download Service Account Key

https://console.cloud.google.com/apis/credentials?project=bootcamp-1569375678077&folder&organizationId

Enable BigQuery API

https://console.developers.google.com/apis/api/bigquery-json.googleapis.com/overview?project=bootcamp-1569375678077

### Enable Google Credentials for every session

GOOGLE_APPLICATION_CREDENTIALS must be set:

https://console.cloud.google.com/home/dashboard?cloudshell=true&_ga=2.36182944.-145041138.1573320821&pli=1&project=bootcamp-1569375678077&folder&organizationId

### Get Data from BigQuery

kaggle Reference for BigQuery API

https://www.kaggle.com/sohier/beyond-queries-exploring-the-bigquery-api

In [1]:
# Constants

# Google service credentials file
service_account_json = "C:/Users/janin/OneDrive/Documents/GitHub/Bootcamp-7917e2b029b4.json"
# BigQueryPublic data file
PROJECT_ID = 'bigquery-public-data' # All BigQuery public datasets use this project ID
dataset = 'census_bureau_international' # Name of the dataset we're interested in

In [2]:
# Dependencies to be able to get data from Google BigQuery
from google.cloud import bigquery
bigquery_client = bigquery.Client(project=PROJECT_ID)
print(bigquery_client)
from google.cloud import storage
storage_client = storage.Client(project=PROJECT_ID)
print(storage_client)

In [3]:
# Get dataset reference
census_dataset_ref = bigquery_client.dataset(dataset, project=PROJECT_ID)
census_dataset_ref

DatasetReference('bigquery-public-data', 'census_bureau_international')

In [4]:
# Get dataset
census_dset = bigquery_client.get_dataset(census_dataset_ref)
census_dset

Dataset(DatasetReference('bigquery-public-data', 'census_bureau_international'))

To resolve  403 error, enabled BigQuery API at https://console.developers.google.com/apis/library/bigquery-json.googleapis.com?project=bootcamp-1569375678077&pli=1.

Example error:

Forbidden: 403 GET https://bigquery.googleapis.com/bigquery/v2/projects/bigquery-public-data/datasets/census_bureau_international: BigQuery API has not been used in project 488912422205 before or it is disabled. Enable it by visiting https://console.developers.google.com/apis/api/bigquery-json.googleapis.com/overview?project=488912422205 then retry. If you enabled this API recently, wait a few minutes for the action to propagate to our systems and retry.

In [5]:
# List tables in the dataset
print("Tables:")
tables = []
for table in bigquery_client.list_tables(census_dset):
    table_id = table.table_id
    if len(table_id)>0:
        tables.append(table_id)
        print(table_id)

Tables:
age_specific_fertility_rates
birth_death_growth_rates
country_names_area
midyear_population
midyear_population_5yr_age_sex
midyear_population_age_sex
midyear_population_agespecific
mortality_life_expectancy


In [7]:
desired_tables  = ['birth_death_growth_rates',
                  'country_names_area',
                  'midyear_population',
                  'mortality_life_expectancy']
print('Tables:')
census_tables = []
for table in desired_tables:
    census_table = bigquery_client.get_table(census_dset.table(table))
    print(census_table.table_id)
    census_tables.append(census_table)
print()
print("Table Commands:")
census_commands = [command for command in dir(census_tables[0]) if not command.startswith('_')]
print(census_commands)
print()
print("Schema Commands:")
schema_commands = [command for command in dir(census_tables[0].schema[0]) if not command.startswith('_')]
print(schema_commands)

Tables:
birth_death_growth_rates
country_names_area
midyear_population
mortality_life_expectancy

Table Commands:
['clustering_fields', 'created', 'dataset_id', 'description', 'encryption_configuration', 'etag', 'expires', 'external_data_configuration', 'friendly_name', 'from_api_repr', 'from_string', 'full_table_id', 'labels', 'location', 'modified', 'num_bytes', 'num_rows', 'partition_expiration', 'partitioning_type', 'path', 'project', 'reference', 'require_partition_filter', 'schema', 'self_link', 'streaming_buffer', 'table_id', 'table_type', 'time_partitioning', 'to_api_repr', 'to_bqstorage', 'view_query', 'view_use_legacy_sql']

Schema Commands:
['description', 'field_type', 'fields', 'from_api_repr', 'is_nullable', 'mode', 'name', 'to_api_repr', 'to_standard_sql']


In [9]:
print('Schemas')
for census_table in census_tables:
    print()
    print('__________________________________________________________________________________________________')
    print(f"Table: {census_table.table_id}")
    print('__________________________________________________________________________________________________')
    for schema_field in census_table.schema:
        print(f"{schema_field.name} {schema_field.field_type} {schema_field.description}")        

Schemas

__________________________________________________________________________________________________
Table: birth_death_growth_rates
__________________________________________________________________________________________________
country_code STRING Federal Information Processing Standard (FIPS) country/area code
country_name STRING Country or area name
year INTEGER Year
crude_birth_rate FLOAT Crude birth rate (births per 1,000 population)
crude_death_rate FLOAT Crude death rate (deaths per 1,000 population)
net_migration FLOAT Net migration rate (net number of migrants per 1,000 population)
rate_natural_increase FLOAT Rate of natural increase (percent)
growth_rate FLOAT Growth rate (percent)

__________________________________________________________________________________________________
Table: country_names_area
__________________________________________________________________________________________________
country_code STRING Federal Information Processing Standard (FIP

In [34]:
# Get data for 2008 to 2016
table_ids = []
countries = []
years = []
columns = []
results = []
for census_table in census_tables:
    schema_cols = [col for col in census_table.schema]
    table_id = census_table.table_id
    for row in bigquery_client.list_rows(census_table, start_index=100, selected_fields=schema_cols):
        row_values = dict(row)
        if "year" in row_values.keys():
            year = row_values["year"]
            country = row_values["country_name"]
            if year>=2008 and year<=2016:
                for col in schema_cols:
                    column = col.name
                    if (column!="year") and (column!="country_name"):
                        table_ids.append(table_id)
                        countries.append(country)
                        years.append(year)
                        columns.append(column)
                        result = row[column]
                        results.append(result)

In [54]:
# Add lists to DataFrame
import pandas as pd
censusdf = pd.DataFrame()
censusdf["Table"] = table_ids
censusdf["Country"] = countries
censusdf["Year"] = years
censusdf["Column"] = columns
censusdf["Value"] = results
censusdf.to_csv("Census_Raw_Data.csv")
#censusdf.set_index(['Table','Country','Year'],inplace=True)
censusdf

,Table,Country,Year,Column,Value
0,birth_death_growth_rates,Fiji,2008,country_code,FJ
1,birth_death_growth_rates,Fiji,2008,crude_birth_rate,23.04
2,birth_death_growth_rates,Fiji,2008,crude_death_rate,5.86
3,birth_death_growth_rates,Fiji,2008,net_migration,-7.66
4,birth_death_growth_rates,Fiji,2008,rate_natural_increase,1.718
5,birth_death_growth_rates,Fiji,2008,growth_rate,0.952
6,birth_death_growth_rates,Fiji,2009,country_code,FJ
7,birth_death_growth_rates,Fiji,2009,crude_birth_rate,22.28
8,birth_death_growth_rates,Fiji,2009,crude_death_rate,5.86
9,birth_death_growth_rates,Fiji,2009,net_migration,-7.51


In [52]:
census_pivot = pd.DataFrame()
tables = []
countries = []
years = []


KeyError: 'Level Table not found'

In [38]:
dfs = []
for table in desired_tables:
    single_table = censusdf.loc[censusdf["Table"]==table].copy()
    print(single_table.head())
    pivot_table = pd.pivot(single_table,values='Value',index=['Table','Country','Year'],columns='Column').reset_index()
    print(pivot_table.head())
    dfs.append(pivot_table)

                      Table Country  Year                 Column  Value
0  birth_death_growth_rates    Fiji  2008           country_code     FJ
1  birth_death_growth_rates    Fiji  2008       crude_birth_rate  23.04
2  birth_death_growth_rates    Fiji  2008       crude_death_rate   5.86
3  birth_death_growth_rates    Fiji  2008          net_migration  -7.66
4  birth_death_growth_rates    Fiji  2008  rate_natural_increase  1.718


ValueError: Length of passed values is 12174, index implies 3